In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch import Tensor
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter

In [2]:
writer = SummaryWriter()

In [3]:
train_set = pd.read_csv("data/final_format/train_set.csv",header=None).to_numpy()
train_label = pd.read_csv("data/final_format/train_label.csv",header=None).to_numpy()
test_set = pd.read_csv("data/final_format/test_set.csv",header=None).to_numpy()
test_label = pd.read_csv("data/final_format/test_label.csv",header=None).to_numpy()

In [4]:
print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

(14393, 4096) (14393, 1) (3599, 4096) (3599, 1)


In [5]:
#delet first row data
train_set = train_set[1:]
train_label = train_label[1:]
test_set = test_set[1:]
test_label = test_label[1:]
print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

(14392, 4096) (14392, 1) (3598, 4096) (3598, 1)


In [6]:
train_set = train_set.reshape((-1,1,64,64))
test_set = test_set.reshape((-1,1,64,64))
print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

(14392, 1, 64, 64) (14392, 1) (3598, 1, 64, 64) (3598, 1)


In [7]:
train_label = train_label.reshape(-1)
test_label = test_label.reshape(-1)

print(train_set.shape, train_label.shape, test_set.shape, test_label.shape)

(14392, 1, 64, 64) (14392,) (3598, 1, 64, 64) (3598,)


In [8]:
# Hyper parameters
num_epochs = 300
num_classes = 4
batch_size = 64
learning_rate = 1e-3

In [9]:
train_set_tensor = Tensor(train_set) 
train_label_tensor = Tensor(train_label).type(torch.LongTensor)

train_dataset = TensorDataset(train_set_tensor,train_label_tensor) 
train_loader = DataLoader(train_dataset, batch_size=batch_size) 

test_set_tensor = Tensor(test_set) 
test_label_tensor = Tensor(test_label).type(torch.LongTensor)

test_dataset = TensorDataset(test_set_tensor,test_label_tensor) 
test_loader = DataLoader(test_dataset, batch_size=batch_size) 

In [10]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [11]:
# Convolutional neural network (two convolutional layers)
class FCN(nn.Module):
    def __init__(self, num_classes=4):
        super(FCN, self).__init__()
        self.c1 = nn.Conv2d(1, 32, kernel_size=3, padding='same')
        self.c2 = nn.Conv2d(32, 32, kernel_size=3,padding='same')
        self.c3 = nn.Conv2d(64, 64, kernel_size=3,padding='same') 
        self.c4 = nn.Conv2d(64, 64, kernel_size=3, padding='same')
        self.c5 = nn.Conv2d(64, 64, kernel_size=3,padding='same')
        self.c6 = nn.Conv2d(128, 128, kernel_size=3, padding='same')
        self.m1 = nn.Conv2d(64, 64, kernel_size=3,stride=2,padding='valid')
        self.m2 = nn.Conv2d(128, 128, kernel_size=3,stride=2,padding='valid')
        
        self.d1 = nn.Dropout(p=0.25)
        self.d2 = nn.Dropout(p=0.25)
        self.d3 = nn.Dropout(p=0.25)
        self.d4 = nn.Dropout(p=0.25)
        self.d5 = nn.Dropout(p=0.25)
     
        self.bn1 =  nn.BatchNorm2d(32)
        self.bn2 =  nn.BatchNorm2d(64)
        self.bn3 =  nn.BatchNorm2d(64)
        self.bn4 =  nn.BatchNorm1d(512)
       
        self.fc1 = nn.Linear(128*15*15, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        
    def forward(self, x):
        C1 = self.c1(x)
        C1 = F.leaky_relu(C1,0.2)
        C1 = self.d1(C1)
        C1 = self.bn1(C1)
        C2 = self.c2(C1)
        C2 = F.leaky_relu(C2,0.2)
        sum1 = torch.cat((C1, C2), dim=1)
        C3 = self.c3(sum1)
        C3 = F.leaky_relu(C3,0.2)
        C3 = self.d2(C3)
        M1 = self.m1(C3)
        
        C4 = self.c4(M1)
        C4 = self.bn2(C4)
        C4 = F.leaky_relu(C4,0.2)
        C4 = self.d3(C4)
        C5 = self.c5(C4)
        C5 = self.bn3(C5)
        C5 = F.leaky_relu(C5,0.2)
        sum2 = torch.cat((C4, C5), dim=1)
        C6 = self.c6(sum2)
        C6 = F.leaky_relu(C6,0.2)
        C6 = self.d4(C6)
        M2 = self.m2(C6)

        F1 = M2.reshape(M2.size(0), -1)
        Fc1 = self.fc1(F1)
        Fc1 = self.bn4(Fc1)
        Fc1 = F.leaky_relu(Fc1,0.2)
        Fc1 = self.d5(Fc1)
        Fc2 = self.fc2(Fc1)
       
        return Fc2
    


In [12]:
model = FCN(num_classes).to(device)

In [13]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3) 
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
milestones = [50,100,150,200,250]
milestones = [a * len(train_loader) for a in milestones]
scheduler = MultiStepLR(optimizer, milestones=milestones, gamma=0.5)

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    correct=0
    total=0
    running_loss = 0
    for i, (X, Y) in enumerate(train_loader):
        X = X.to(device)
        Y = Y.to(device)
        
        
        # Forward pass
        outputs = model(X)
        loss = criterion(outputs, Y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        #scheduler.step() 
        #print(scheduler.get_last_lr()[0])
      
        optimizer.step()
        scheduler.step() 
        #print(optimizer.param_groups[0]["lr"])
        
        _, predicted = outputs.max(1)
        total += Y.size(0)
        correct += predicted.eq(Y).sum().item()
        running_loss += loss.item()
        accu=100.*correct/total
        train_loss = running_loss/(i+1)
        print ('Epoch [{}/{}], Step [{}/{}], Training Accuracy: {:.4f}%, Training Loss: {:.4f}%'.format(epoch+1, num_epochs, i+1, total_step, accu, train_loss))
    
   
        #writer.add_scalar(f'train/accuracy', accu, epoch)
        #writer.add_scalar(f'train/loss', train_loss, epoch)
        writer.add_scalars(f'train/accuracy_loss', {
            'accuracy': accu,
            'loss': train_loss,
        }, epoch)
        
        

Epoch [1/300], Step [1/225], Training Accuracy: 26.5625%, Training Loss: 1.4490%
Epoch [1/300], Step [2/225], Training Accuracy: 24.2188%, Training Loss: 2.0552%
Epoch [1/300], Step [3/225], Training Accuracy: 27.6042%, Training Loss: 1.9193%
Epoch [1/300], Step [4/225], Training Accuracy: 30.4688%, Training Loss: 1.8943%
Epoch [1/300], Step [5/225], Training Accuracy: 30.3125%, Training Loss: 1.8225%
Epoch [1/300], Step [6/225], Training Accuracy: 29.9479%, Training Loss: 1.7859%
Epoch [1/300], Step [7/225], Training Accuracy: 29.4643%, Training Loss: 1.7332%
Epoch [1/300], Step [8/225], Training Accuracy: 29.1016%, Training Loss: 1.6994%
Epoch [1/300], Step [9/225], Training Accuracy: 28.9931%, Training Loss: 1.6685%
Epoch [1/300], Step [10/225], Training Accuracy: 28.1250%, Training Loss: 1.6538%
Epoch [1/300], Step [11/225], Training Accuracy: 27.9830%, Training Loss: 1.6354%
Epoch [1/300], Step [12/225], Training Accuracy: 27.7344%, Training Loss: 1.6115%
Epoch [1/300], Step [13/2

Epoch [1/300], Step [104/225], Training Accuracy: 30.6190%, Training Loss: 1.4085%
Epoch [1/300], Step [105/225], Training Accuracy: 30.5804%, Training Loss: 1.4086%
Epoch [1/300], Step [106/225], Training Accuracy: 30.6751%, Training Loss: 1.4076%
Epoch [1/300], Step [107/225], Training Accuracy: 30.7243%, Training Loss: 1.4072%
Epoch [1/300], Step [108/225], Training Accuracy: 30.8304%, Training Loss: 1.4067%
Epoch [1/300], Step [109/225], Training Accuracy: 30.7626%, Training Loss: 1.4064%
Epoch [1/300], Step [110/225], Training Accuracy: 30.8381%, Training Loss: 1.4052%
Epoch [1/300], Step [111/225], Training Accuracy: 30.8136%, Training Loss: 1.4051%
Epoch [1/300], Step [112/225], Training Accuracy: 30.8454%, Training Loss: 1.4045%
Epoch [1/300], Step [113/225], Training Accuracy: 30.8075%, Training Loss: 1.4055%
Epoch [1/300], Step [114/225], Training Accuracy: 30.9073%, Training Loss: 1.4042%
Epoch [1/300], Step [115/225], Training Accuracy: 30.9783%, Training Loss: 1.4032%
Epoc

Epoch [1/300], Step [208/225], Training Accuracy: 32.3017%, Training Loss: 1.3711%
Epoch [1/300], Step [209/225], Training Accuracy: 32.3041%, Training Loss: 1.3712%
Epoch [1/300], Step [210/225], Training Accuracy: 32.2991%, Training Loss: 1.3711%
Epoch [1/300], Step [211/225], Training Accuracy: 32.2793%, Training Loss: 1.3706%
Epoch [1/300], Step [212/225], Training Accuracy: 32.2745%, Training Loss: 1.3701%
Epoch [1/300], Step [213/225], Training Accuracy: 32.2917%, Training Loss: 1.3699%
Epoch [1/300], Step [214/225], Training Accuracy: 32.3379%, Training Loss: 1.3695%
Epoch [1/300], Step [215/225], Training Accuracy: 32.3038%, Training Loss: 1.3695%
Epoch [1/300], Step [216/225], Training Accuracy: 32.2772%, Training Loss: 1.3696%
Epoch [1/300], Step [217/225], Training Accuracy: 32.3085%, Training Loss: 1.3687%
Epoch [1/300], Step [218/225], Training Accuracy: 32.3179%, Training Loss: 1.3686%
Epoch [1/300], Step [219/225], Training Accuracy: 32.3701%, Training Loss: 1.3679%
Epoc

Epoch [2/300], Step [88/225], Training Accuracy: 36.7543%, Training Loss: 1.2924%
Epoch [2/300], Step [89/225], Training Accuracy: 36.6397%, Training Loss: 1.2929%
Epoch [2/300], Step [90/225], Training Accuracy: 36.7361%, Training Loss: 1.2926%
Epoch [2/300], Step [91/225], Training Accuracy: 36.7960%, Training Loss: 1.2919%
Epoch [2/300], Step [92/225], Training Accuracy: 36.8376%, Training Loss: 1.2918%
Epoch [2/300], Step [93/225], Training Accuracy: 36.9792%, Training Loss: 1.2908%
Epoch [2/300], Step [94/225], Training Accuracy: 37.0346%, Training Loss: 1.2890%
Epoch [2/300], Step [95/225], Training Accuracy: 36.9737%, Training Loss: 1.2903%
Epoch [2/300], Step [96/225], Training Accuracy: 37.0605%, Training Loss: 1.2888%
Epoch [2/300], Step [97/225], Training Accuracy: 37.1295%, Training Loss: 1.2888%
Epoch [2/300], Step [98/225], Training Accuracy: 37.0376%, Training Loss: 1.2902%
Epoch [2/300], Step [99/225], Training Accuracy: 37.0265%, Training Loss: 1.2899%
Epoch [2/300], S

Epoch [2/300], Step [192/225], Training Accuracy: 38.4277%, Training Loss: 1.2604%
Epoch [2/300], Step [193/225], Training Accuracy: 38.3986%, Training Loss: 1.2603%
Epoch [2/300], Step [194/225], Training Accuracy: 38.4101%, Training Loss: 1.2598%
Epoch [2/300], Step [195/225], Training Accuracy: 38.4535%, Training Loss: 1.2595%
Epoch [2/300], Step [196/225], Training Accuracy: 38.4327%, Training Loss: 1.2596%
Epoch [2/300], Step [197/225], Training Accuracy: 38.4280%, Training Loss: 1.2595%
Epoch [2/300], Step [198/225], Training Accuracy: 38.5417%, Training Loss: 1.2580%
Epoch [2/300], Step [199/225], Training Accuracy: 38.5914%, Training Loss: 1.2569%
Epoch [2/300], Step [200/225], Training Accuracy: 38.6016%, Training Loss: 1.2566%
Epoch [2/300], Step [201/225], Training Accuracy: 38.5805%, Training Loss: 1.2566%
Epoch [2/300], Step [202/225], Training Accuracy: 38.5907%, Training Loss: 1.2567%
Epoch [2/300], Step [203/225], Training Accuracy: 38.5622%, Training Loss: 1.2564%
Epoc

...

Epoch [49/300], Step [140/225], Training Accuracy: 76.3504%, Training Loss: 0.5240%
Epoch [49/300], Step [141/225], Training Accuracy: 76.3741%, Training Loss: 0.5235%
Epoch [49/300], Step [142/225], Training Accuracy: 76.3644%, Training Loss: 0.5236%
Epoch [49/300], Step [143/225], Training Accuracy: 76.3658%, Training Loss: 0.5231%
Epoch [49/300], Step [144/225], Training Accuracy: 76.3563%, Training Loss: 0.5228%
Epoch [49/300], Step [145/225], Training Accuracy: 76.3470%, Training Loss: 0.5228%
Epoch [49/300], Step [146/225], Training Accuracy: 76.3592%, Training Loss: 0.5227%
Epoch [49/300], Step [147/225], Training Accuracy: 76.3499%, Training Loss: 0.5226%
Epoch [49/300], Step [148/225], Training Accuracy: 76.4358%, Training Loss: 0.5215%
Epoch [49/300], Step [149/225], Training Accuracy: 76.4367%, Training Loss: 0.5211%
Epoch [49/300], Step [150/225], Training Accuracy: 76.4167%, Training Loss: 0.5213%
Epoch [49/300], Step [151/225], Training Accuracy: 76.4797%, Training Loss: 

Epoch [50/300], Step [14/225], Training Accuracy: 78.5714%, Training Loss: 0.4891%
Epoch [50/300], Step [15/225], Training Accuracy: 78.8542%, Training Loss: 0.4829%
Epoch [50/300], Step [16/225], Training Accuracy: 79.1016%, Training Loss: 0.4777%
Epoch [50/300], Step [17/225], Training Accuracy: 78.7684%, Training Loss: 0.4798%
Epoch [50/300], Step [18/225], Training Accuracy: 78.7326%, Training Loss: 0.4849%
Epoch [50/300], Step [19/225], Training Accuracy: 78.2072%, Training Loss: 0.4885%
Epoch [50/300], Step [20/225], Training Accuracy: 78.2031%, Training Loss: 0.4879%
Epoch [50/300], Step [21/225], Training Accuracy: 78.4226%, Training Loss: 0.4831%
Epoch [50/300], Step [22/225], Training Accuracy: 78.0540%, Training Loss: 0.4883%
Epoch [50/300], Step [23/225], Training Accuracy: 78.0571%, Training Loss: 0.4881%
Epoch [50/300], Step [24/225], Training Accuracy: 77.9297%, Training Loss: 0.4937%
Epoch [50/300], Step [25/225], Training Accuracy: 78.0000%, Training Loss: 0.4917%
Epoc

Epoch [50/300], Step [114/225], Training Accuracy: 77.2341%, Training Loss: 0.5081%
Epoch [50/300], Step [115/225], Training Accuracy: 77.3370%, Training Loss: 0.5072%
Epoch [50/300], Step [116/225], Training Accuracy: 77.3303%, Training Loss: 0.5069%
Epoch [50/300], Step [117/225], Training Accuracy: 77.3104%, Training Loss: 0.5074%
Epoch [50/300], Step [118/225], Training Accuracy: 77.2643%, Training Loss: 0.5078%
Epoch [50/300], Step [119/225], Training Accuracy: 77.2059%, Training Loss: 0.5085%
Epoch [50/300], Step [120/225], Training Accuracy: 77.2266%, Training Loss: 0.5080%
Epoch [50/300], Step [121/225], Training Accuracy: 77.1694%, Training Loss: 0.5088%
Epoch [50/300], Step [122/225], Training Accuracy: 77.1004%, Training Loss: 0.5097%
Epoch [50/300], Step [123/225], Training Accuracy: 77.0960%, Training Loss: 0.5099%
Epoch [50/300], Step [124/225], Training Accuracy: 77.0413%, Training Loss: 0.5102%
Epoch [50/300], Step [125/225], Training Accuracy: 77.0875%, Training Loss: 

Epoch [50/300], Step [214/225], Training Accuracy: 77.8768%, Training Loss: 0.5013%
Epoch [50/300], Step [215/225], Training Accuracy: 77.8561%, Training Loss: 0.5018%
Epoch [50/300], Step [216/225], Training Accuracy: 77.8573%, Training Loss: 0.5018%
Epoch [50/300], Step [217/225], Training Accuracy: 77.8226%, Training Loss: 0.5025%
Epoch [50/300], Step [218/225], Training Accuracy: 77.7881%, Training Loss: 0.5026%
Epoch [50/300], Step [219/225], Training Accuracy: 77.7897%, Training Loss: 0.5028%
Epoch [50/300], Step [220/225], Training Accuracy: 77.8054%, Training Loss: 0.5028%
Epoch [50/300], Step [221/225], Training Accuracy: 77.7856%, Training Loss: 0.5031%
Epoch [50/300], Step [222/225], Training Accuracy: 77.7872%, Training Loss: 0.5028%
Epoch [50/300], Step [223/225], Training Accuracy: 77.8027%, Training Loss: 0.5026%
Epoch [50/300], Step [224/225], Training Accuracy: 77.8181%, Training Loss: 0.5024%
Epoch [50/300], Step [225/225], Training Accuracy: 77.8141%, Training Loss: 

Epoch [51/300], Step [88/225], Training Accuracy: 79.1016%, Training Loss: 0.4690%
Epoch [51/300], Step [89/225], Training Accuracy: 79.1959%, Training Loss: 0.4681%
Epoch [51/300], Step [90/225], Training Accuracy: 79.1319%, Training Loss: 0.4689%
Epoch [51/300], Step [91/225], Training Accuracy: 79.1380%, Training Loss: 0.4689%
Epoch [51/300], Step [92/225], Training Accuracy: 79.0761%, Training Loss: 0.4699%
Epoch [51/300], Step [93/225], Training Accuracy: 79.2003%, Training Loss: 0.4684%
Epoch [51/300], Step [94/225], Training Accuracy: 79.1888%, Training Loss: 0.4682%
Epoch [51/300], Step [95/225], Training Accuracy: 79.1776%, Training Loss: 0.4688%
Epoch [51/300], Step [96/225], Training Accuracy: 79.2480%, Training Loss: 0.4675%
Epoch [51/300], Step [97/225], Training Accuracy: 79.2848%, Training Loss: 0.4673%
Epoch [51/300], Step [98/225], Training Accuracy: 79.2411%, Training Loss: 0.4680%
Epoch [51/300], Step [99/225], Training Accuracy: 79.2929%, Training Loss: 0.4676%
Epoc

In [ ]:
# Test the model

model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for X, Y in test_loader:
        X = X.to(device)
        Y = Y.to(device)
        outputs = model(X)
        _, predicted = torch.max(outputs.data, 1)
        total += Y.size(0)
        correct += (predicted == Y).sum().item()

    print('Test Accuracy : {} %'.format(100 * correct / total))

# Save the model checkpoint
#torch.save(model.state_dict(), 'model.ckpt')